# Resource Assignment Problem formulation

Consider three job positions: Tester, Java-Developer, and Architect.

Consider three resources: Carlos, Joe, and Monika.

## Data 

The ability to perform each of the jobs by each of the resources is illustrated by the following matching scores table:

![Resource Allocation Problem Data Image](util/rap_data.png)


**Assumption**: Only one resource can be assigned to a job, and only one job can be assigned to a resource.

## Problem statement

Determine an assignment that ensures that each job is fulfilled and each resource is assigned to at most one job in order to maximize the total matching scores of the assignments.

## Decision variables

The decision variable $x_{r,\; j} = 1$ represents that resource r is assigned to job j, and 0 otherwise, for  r=1,2,3 and 𝑗=1,2,3.

## Constraints

### Jobs constraints

For each job 𝑗=1,2,3, exactly one resource from r=1,2,3 must be assigned.

Constraint (Tester=1): $x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

Constraint (Java-Developer=2): $x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

Constraint (Architect=3): $x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

### Resources constraints

For each resource = r=1,2,3, at most one job from r=1,2,3 can be assigned.

Constraint (Carlos=1): $x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

Constraint (Joe=2): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

Constraint (Monika=3): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

## Objective function

The objective function is to maximize the total matching score of the assignments while satisfying the jobs and resources constraints.

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$




First, let's install gurobipy as needed

In [ ]:
%pip install gurobipy

In [2]:
# import gurobi library
from gurobipy import *

## Data
The list R contains the names of the three resources: Carlos, Joe, and Monika. 

The list J contains the names of the job positions: tester, java-developer, and architect.

**Math notation**

$r \in R$ means that a resource with index r is in the set (list) R.

$j \in J$ means that a job with index j is in the set (list) J.

In [3]:
# resources and jobs sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

The following “multidict” function describes the matching score associated with each possible combination of a resource and job.

**Math notation**

Let $ms_{r,\;j}$ be the matching score of resource  $r \in R$  with respect to job  $j \in J$.

Let $C_{r,\;j}$ be the cost of assigning resource  $r \in R$  to job  $j \in J$.

Let $B$ be the budget available.

In [4]:
# matching score data
combinations, ms, C = multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# Budget available
#B = 6
B=5

The following function generates an empty model object “m” and takes the string “RAP” model name as its argument.

In [5]:
# Declare and initialize model
m = Model('RAP')

## Decision variables

The decision variable $x_{r,\; j} = 1$ represents that resource r is assigned to job j, and 0 otherwise, for  r=1,2,3 and 𝑗=1,2,3.

The “addVars()” method defines the decision variables of the model object “m”.  

**Math notation**

Let $x_{r,\; j} = 1$ if resource $r \in R$  is assigend to job $j \in J$, and zero otherwise.

Let $g_{j} = 1$ if job $j \in J$ cannot be filled, and zero otherwise. This variable is a gap variable that indicates that a job cannot be filled.


In [6]:
# Create decision variables for the RAP model
#x = m.addVars(combinations, name="assign")
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# Create gap variables for the RAP model
g = m.addVars(J, name="gap")

## Jobs constraints

For each job 𝑗=1,2,3, exactly one resource from r=1,2,3 must be assigned.

Constraint (Tester=1): $x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} + g_{1} = 1$

Constraint (Java-Developer=2): $x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} + g_{2} = 1$

Constraint (Architect=3): $x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} + g_{3} = 1$

The “addConstrs()” method defines the constraints of the model object “m”. 

**Math notation**

For each job $j \in J$, exactly one resouce must be assigned:

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1 
$$



In [7]:
# create jobs  constraints
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), 'job')

## Resources constraints

For each resource = r=1,2,3, at most one job from r=1,2,3 can be assigned.

Constraint (Carlos=1): $x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

Constraint (Joe=2): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

Constraint (Monika=3): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

The “addConstrs()” method defines the constraints of the model object “m”. 

**Math notation**

For each resource $r \in R$, at most one job can be assigned:

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1 
$$

In [8]:
# create resources constraints
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), 'resource')

## Budget constraint

The total cost of assigning resources to jobs should be less or equal than the budget available.

$$
\sum_{r \; \in \; R} \sum_{j \; \in \; J} C_{r, j}x_{r, j} \leq B
$$

In [9]:
budget = m.addConstr((x.prod(C) <= B), 'budget')

## Objective Function

The objective function is to maximize the total matching score of the assignments.

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$

The “setObjective()” method defines the objective function of the model object “m”. 

**Math notation**

Notice that 
$$
(53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

Hence, the objective function can be expressed as follows

$$
Max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j} -BigM \sum_{j \in J} g_{j}
$$


In [10]:
# Penalty for not filling a job position
BIGM =101

# The objective is to maximize total matching score of the assignments
m.setObjective(x.prod(ms) -BIGM*g.sum(), GRB.MAXIMIZE)

In [11]:
# save model for inspection
m.write('RAP3.lp')

In [12]:
# run optimization engine
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xa1231a12
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160%     -    0s
     0     0     cutoff    0  

In [13]:
# display optimal values of decision variables
for v in m.getVars():
	if (abs(v.x) > 1e-6):
		print(v.varName, v.x)

# display optimal total matching score
print('Optimal objective function value', m.objVal)   

# Compute total matching score from assignment  variables
total_matching_score = 0
for [r, j] in combinations:
    if (abs(x[r, j].x) > 1e-6):
        total_matching_score = total_matching_score + ms[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score)  


assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
gap[Architect] 1.0
Optimal objective function value 52.0
Total matching score:  153.0
